In [1]:
from brainpipe.system import study
from itertools import combinations, product
import numpy as np
from os.path import join, exists
from os import makedirs
from scipy import stats
from joblib import Parallel, delayeda

In [14]:
"""
Compute TPSim BTW 2 different odors
Compute similarity matrices for all odors
"""
###############################################################################
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_path = join(path_pow, '{}_odor_{}_bipo_all_{}_6freqs.npz')
path2save = join(st.path, 'feature/TPSim_3groups_{}/similarity_matrix_btw_v=1_elecs=all/')
tps_save = join(path2save, 'TPS_{}_{}_{}_btw_odors_mean.npz')
###############################################################################
exp = 'Enc' #Ret, Enc
stat = 'pearson'
#shortcut = ['E','R'] if exp == 'Enc_Ret' else None
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs, f = ['theta'], 1
odors_su = {'CHAF': [1,2,4,5,3,8,7,9], #low than high
            'LEFC': [15,2,1,16,14,3,4,17],
            'PIRJ': [1,9,5,4,6,7,18], #missing odor 15
            'VACJ': [11,14,12,10,15,17,16,13],
            'SEMC': [7,10,11,12,13,5,8,9],
            'FERJ': [7,2,16,17,12,1,5,13]}

def tpsim_btw_2odors(pow_o1,pow_o2,stat='pearson',average=True):
    """
    Compute tpsim between 2 odors 
    Parameters
    ----------
    pow_o1, pow_o2 : array (npts x ntrials)
    stat : string
            The stat correlation method to use. 'pearson' or 'spearman'
    Returns
    -------
    tpsim : array (Average TPSim between 2 ODORS)
    """
    
    corr = stats.pearsonr if stat == 'pearson' else stats.spearmanr
    sim_trials = np.array([])
    for t0, t1 in product(range(pow_o1.shape[-1]),range(pow_o2.shape[-1])):
        R, _ = corr(pow_o1[:,t0],pow_o2[:,t1])
        sim_trials = np.vstack((sim_trials,1-R)) if np.size(sim_trials) else 1-R
    if average == True:
        sim_trials = np.mean(sim_trials)
    return sim_trials
    
for su,freq in product(odors_su,freqs):
    ncomb = len([c for c in product(odors_su[su],odors_su[su])])
    od_sel = list(odors_su[su])[0]
    mat_ = np.load(pow_path.format(su,str(od_sel),exp[0],freq),allow_pickle=True)
    print(mat_.files)
    chans, labels, xyz = mat_['channels'], mat_['labels'], mat_['xyz']
    nelecs = mat_['xpow'].shape[0]
    tps = np.empty((nelecs,ncomb))
    list_comb = np.array([])
    for i,comb in enumerate(product(odors_su[su],odors_su[su])):
        print(su,freq,comb)
        mat_o1 = np.load(pow_path.format(su,comb[0],exp[0],freq),allow_pickle=True)
        pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
        mat_o2 = np.load(pow_path.format(su,comb[1],exp[0],freq),allow_pickle=True)
        pow_o2 = mat_o2['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
#         pow_o1, pow_o2 = pow_o1.astype('float'), pow_o2.astype('float')
        print(pow_o1.shape,pow_o2.shape)
        list_comb = np.vstack((list_comb,comb)) if np.size(list_comb) else comb
#         Parallel(n_jobs=-1)(delayed(tpsim_btw_2odors)
#             (tps,i,elec,pow_o1[elec],pow_o2[elec],average=True) for elec in range(nelecs))
        
        for elec in range(nelecs):
            if comb[0] != comb[1]: #different odors
                tps[elec,i] = tpsim_btw_2odors(pow_o1[elec],pow_o2[elec],average=True)
            else:
                if pow_o1.shape[-1] == 1: #similarity = 1 for one trial 
                    tps[elec,i] = 0
                else: #compute similiarities across repetitions of the same odor
                    tps[elec,i] = tpsim_by_cond(pow_o1[elec],average=True)
    np.savez(tps_save.format(exp,stat,su,freq),tps=tps,comb=list_comb,channels=chans,
                labels=labels, xyz=xyz)

-> Olfacto loaded
['f', 'split', 'width', 'step', 'time', 'fname', 'labels', 'channels', 'xyz', 'xpow', 'xpow_pval']
CHAF theta (1, 1)
(61, 30, 1) (61, 30, 1)
CHAF theta (1, 2)
(61, 30, 1) (61, 30, 3)
CHAF theta (1, 4)
(61, 30, 1) (61, 30, 1)
CHAF theta (1, 5)
(61, 30, 1) (61, 30, 1)
CHAF theta (1, 3)
(61, 30, 1) (61, 30, 3)
CHAF theta (1, 8)
(61, 30, 1) (61, 30, 3)
CHAF theta (1, 7)
(61, 30, 1) (61, 30, 4)
CHAF theta (1, 9)
(61, 30, 1) (61, 30, 5)
CHAF theta (2, 1)
(61, 30, 3) (61, 30, 1)
CHAF theta (2, 2)
(61, 30, 3) (61, 30, 3)
CHAF theta (2, 4)
(61, 30, 3) (61, 30, 1)
CHAF theta (2, 5)
(61, 30, 3) (61, 30, 1)
CHAF theta (2, 3)
(61, 30, 3) (61, 30, 3)
CHAF theta (2, 8)
(61, 30, 3) (61, 30, 3)
CHAF theta (2, 7)
(61, 30, 3) (61, 30, 4)
CHAF theta (2, 9)
(61, 30, 3) (61, 30, 5)
CHAF theta (4, 1)
(61, 30, 1) (61, 30, 1)
CHAF theta (4, 2)
(61, 30, 1) (61, 30, 3)
CHAF theta (4, 4)
(61, 30, 1) (61, 30, 1)
CHAF theta (4, 5)
(61, 30, 1) (61, 30, 1)
CHAF theta (4, 3)
(61, 30, 1) (61, 30, 3)
C

VACJ theta (15, 15)
(39, 30, 2) (39, 30, 2)
VACJ theta (15, 17)
(39, 30, 2) (39, 30, 2)
VACJ theta (15, 16)
(39, 30, 2) (39, 30, 3)
VACJ theta (15, 13)
(39, 30, 2) (39, 30, 4)
VACJ theta (17, 11)
(39, 30, 2) (39, 30, 3)
VACJ theta (17, 14)
(39, 30, 2) (39, 30, 3)
VACJ theta (17, 12)
(39, 30, 2) (39, 30, 2)
VACJ theta (17, 10)
(39, 30, 2) (39, 30, 3)
VACJ theta (17, 15)
(39, 30, 2) (39, 30, 2)
VACJ theta (17, 17)
(39, 30, 2) (39, 30, 2)
VACJ theta (17, 16)
(39, 30, 2) (39, 30, 3)
VACJ theta (17, 13)
(39, 30, 2) (39, 30, 4)
VACJ theta (16, 11)
(39, 30, 3) (39, 30, 3)
VACJ theta (16, 14)
(39, 30, 3) (39, 30, 3)
VACJ theta (16, 12)
(39, 30, 3) (39, 30, 2)
VACJ theta (16, 10)
(39, 30, 3) (39, 30, 3)
VACJ theta (16, 15)
(39, 30, 3) (39, 30, 2)
VACJ theta (16, 17)
(39, 30, 3) (39, 30, 2)
VACJ theta (16, 16)
(39, 30, 3) (39, 30, 3)
VACJ theta (16, 13)
(39, 30, 3) (39, 30, 4)
VACJ theta (13, 11)
(39, 30, 4) (39, 30, 3)
VACJ theta (13, 14)
(39, 30, 4) (39, 30, 3)
VACJ theta (13, 12)
(39, 30, 4) 

In [15]:
"""
Compute TPSim BTW 2 different odors
Compute similarity matrices BY ODORS MEMORY GROUP
"""
###############################################################################
from utils import odor_groups_3wgth
st = study('Olfacto')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor_all_elecsFT/')
pow_path = join(path_pow, '{}_odor_{}_bipo_all_{}_6freqs.npz')
path2save = join(st.path, 'feature/TPSim_3groups_{}/similarity_matrix_btw_v=1_elecs=all/')
tps_save = join(path2save, 'TPS_{}_{}_{}_{}_btw_odors_mean.npz')
###############################################################################
exp = 'Enc' #Ret, Enc
stat = 'pearson'
#shortcut = ['E','R'] if exp == 'Enc_Ret' else None
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs, f = ['theta'], 1

def tpsim_btw_2odors(pow_o1,pow_o2,stat='pearson',average=True):
    """
    Compute tpsim between 2 odors 
    Parameters
    ----------
    pow_o1, pow_o2 : array (npts x ntrials)
    stat : string
            The stat correlation method to use. 'pearson' or 'spearman'
    Returns
    -------
    tpsim : array (Average TPSim between 2 ODORS)
    """
    
    corr = stats.pearsonr if stat == 'pearson' else stats.spearmanr
    sim_trials = np.array([])
    for t0, t1 in product(range(pow_o1.shape[-1]),range(pow_o2.shape[-1])):
        R, _ = corr(pow_o1[:,t0],pow_o2[:,t1])
        sim_trials = np.vstack((sim_trials,1-R)) if np.size(sim_trials) else 1-R
    if average == True:
        sim_trials = np.mean(sim_trials)
    return sim_trials

for su,freq in product(odor_groups_3wgth,freqs):
    for cond in odor_groups_3wgth[su]:
        ncomb = len([c for c in product(odor_groups_3wgth[su][cond],odor_groups_3wgth[su][cond])])
        od_sel = list(odor_groups_3wgth[su][cond])[0]
        mat_ = np.load(pow_path.format(su,str(od_sel),exp[0],freq),allow_pickle=True)
        chans, labels, xyz = mat_['channels'], mat_['labels'], mat_['xyz']
        nelecs = mat_['xpow'].shape[0]
        tps = np.empty((nelecs,ncomb))
        list_comb = np.array([])
        for i,comb in enumerate(product(odor_groups_3wgth[su][cond],odor_groups_3wgth[su][cond])):
            print(su,freq,comb)
            mat_o1 = np.load(pow_path.format(su,comb[0],exp[0],freq),allow_pickle=True)
            pow_o1 = mat_o1['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
            mat_o2 = np.load(pow_path.format(su,comb[1],exp[0],freq),allow_pickle=True)
            pow_o2 = mat_o2['xpow'][f,:,17:47,:] #nelecs,npts,ntrials
            print(pow_o1.shape,pow_o2.shape)
            list_comb = np.vstack((list_comb,comb)) if np.size(list_comb) else comb

            for elec in range(nelecs):
                if comb[0] != comb[1]: #different odors
                    tps[elec,i] = tpsim_btw_2odors(pow_o1[elec],pow_o2[elec],average=True)
                else:
                    if pow_o1.shape[-1] == 1: #similarity = 1 for one trial 
                        tps[elec,i] = 0
                    else: #compute similiarities across repetitions of the same odor
                        tps[elec,i] = tpsim_by_cond(pow_o1[elec],average=True)
        np.savez(tps_save.format(exp,stat,su,cond,freq),tps=tps,comb=list_comb,channels=chans,
                    labels=labels, xyz=xyz)

-> Olfacto loaded
CHAF theta ('1', '1')
(61, 30, 1) (61, 30, 1)
CHAF theta ('1', '2')
(61, 30, 1) (61, 30, 3)
CHAF theta ('1', '4')
(61, 30, 1) (61, 30, 1)
CHAF theta ('2', '1')
(61, 30, 3) (61, 30, 1)
CHAF theta ('2', '2')
(61, 30, 3) (61, 30, 3)
CHAF theta ('2', '4')
(61, 30, 3) (61, 30, 1)
CHAF theta ('4', '1')
(61, 30, 1) (61, 30, 1)
CHAF theta ('4', '2')
(61, 30, 1) (61, 30, 3)
CHAF theta ('4', '4')
(61, 30, 1) (61, 30, 1)
CHAF theta ('5', '5')
(61, 30, 1) (61, 30, 1)
CHAF theta ('5', '8')
(61, 30, 1) (61, 30, 3)
CHAF theta ('8', '5')
(61, 30, 3) (61, 30, 1)
CHAF theta ('8', '8')
(61, 30, 3) (61, 30, 3)
CHAF theta ('3', '3')
(61, 30, 3) (61, 30, 3)
CHAF theta ('3', '7')
(61, 30, 3) (61, 30, 4)
CHAF theta ('3', '9')
(61, 30, 3) (61, 30, 5)
CHAF theta ('7', '3')
(61, 30, 4) (61, 30, 3)
CHAF theta ('7', '7')
(61, 30, 4) (61, 30, 4)
CHAF theta ('7', '9')
(61, 30, 4) (61, 30, 5)
CHAF theta ('9', '3')
(61, 30, 5) (61, 30, 3)
CHAF theta ('9', '7')
(61, 30, 5) (61, 30, 4)
CHAF theta ('9',

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from sklearn.manifold import MDS
"""
Plot TPSim matrices (RDM) for all odors by subjects
"""
###############################################################################
st = study('Olfacto')
path_npz = join(st.path,'feature/TPSim_3groups_{}/')
path_pow = join(path_npz, 'similarity_matrix_btw_v=1_elecs=all/TPS_spear_{}_{}_btw_odors_mean.npz')
savename = join(path_npz, 'distance_graphs/{}_{}_{}_{}_{}.png')
###############################################################################
exp = 'Enc' #Ret, Enc
shortcut = ['E']
###############################################################################
freqs = ['0_theta']
su_odor_groups = {'CHAF' : {'low':['1','2','4','5'],
                              'high':['3','8','7','9']},
                    'VACJ' : {'low':['11','14','12','10'],
                              'high':['15','17','16','13']},
                    'SEMC' : {'low':['7','10','11','12','13'],
                              'high':['5','8','9']},
                    'PIRJ' : {'low':['1','9','5'],
                              'high':['4','6','7','18']},
                    'LEFC' : {'low':['15','2','1','16'],
                              'high':['14','3','4','17']},
                    'FERJ' : {'low':['7','2','16','17'],
                              'high':['12','1','5','13']},}
context_su = {'CHAF': {'M':[5,7,8,9],'F':[1,2,3,4]},
              'LEFC': {'M':[1,2,3,4],'F':[14,15,16,17]},
              'PIRJ': {'M':[4,9,1,18],'F':[6,5,7]}, #missing odor 15
              'VACJ': {'M':[14,15,16,17],'F':[10,11,12,13]},
              'SEMC': {'M':[10,11,12,13],'F':[5,7,8,9]},
              'FERJ': {'M':[16,17,5,7],'F':[12,13,2,1]},}

for su,freq in product(su_odor_groups,freqs):
    mat = np.load(path_pow.format(exp,su,freq),allow_pickle=True)
    combs, tps, labels = mat['comb'], mat['tps'], mat['labels']
    
    nelecs, ncombs = tps.shape
    
    for elec in range(nelecs):
        lab = labels[elec]
        n_od = len(np.unique(combs))
        idx = list([combs[0,0]])+list([o for o in combs[:n_od-1,1]])
        tri = np.zeros((n_od, n_od))
        tri[np.triu_indices(n_od, 1)] = tps[elec,:]
        tri[np.tril_indices(n_od, -1)] = tri.T[np.tril_indices(n_od, -1)]

        model = MDS(n_components=2, dissimilarity='precomputed', random_state=1)
        out = model.fit_transform(tri)
        fig, ax = plt.subplots()
        colors = ['red' if str(c) in su_odor_groups[su]['high'] else 'blue' for c in idx]
        markers = ['o' if c in context_su[su]['M'] else '*' for c in idx]

        for i, txt in enumerate(idx):
            ax.scatter(out[i,0], out[i,1], c=colors[i], marker=markers[i],
                       label=['Rich' if colors[i]=='red' else 'Poor'])
            ax.annotate('O'+str(txt), (out[i,0], out[i,1]))
        plt.xlabel('component 1')
        plt.ylabel('component 2')
        plt.axis('equal')

        legend_elements = [Line2D([0], [0], marker='o', color='b', label='Poor_M'),
                           Line2D([0], [0], marker='*', color='b', label='Poor_C'),
                           Line2D([0], [0], marker='o', color='r', label='High_M'),
                           Line2D([0], [0], marker='*', color='r', label='High_C')]
        ax.legend(handles=legend_elements, loc='best')
        title = 'Distance btw odors in {}\n {} at {} in {} elec({})'.format(freq[2:], su,
                                                        exp[0],lab,str(elec))
        fig.suptitle(title, fontsize=12)
        plt.savefig(savename.format(exp,su,freq,exp[0],lab,str(elec)))
        plt.clf()
        plt.close()


In [9]:
from scipy.stats import pearsonr, spearmanr

def tpsim_by_cond(pow1, stat='pearson',average=False):
    """
    Compute tpsim within one condition for all combinations
    Parameters
    ----------
    pow1 : array 
        Must be of shape (npts x ntrials)
    stat : string
        The stat correlation method to use. 'pearson' or 'spearman'
    Returns
    -------
    tpsim : array
    """
    
    corr = pearsonr if stat == 'pearson' else spearmanr
    
    sim_trials = np.array([])
    for t0, t1 in combinations(np.arange(pow1.shape[-1]),2):
        R, _ = corr(pow1[:,t0],pow1[:,t1])
        sim_trials = np.vstack((sim_trials,1-R)) if np.size(sim_trials) else 1-R
    if np.size(sim_trials) == 1:
        sim_trials = np.array([[sim_trials]])
    else:
        sim_trials = sim_trials.swapaxes(0,1)
    if average == True:
        sim_trials = np.mean(sim_trials)
    return sim_trials


In [ ]:
"""
Compute Distance of odors within Poor and Rich conds separately through E and R
Define time windows during E and R --> Dynamics of TPSim
"""
###############################################################################
st = study('Olfacto')
path_tps = join(st.path, 'feature/TPSim_{}_By_Odor_By_Cond/')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor/')
pow_path = join(path_pow, '{}_odor{}_{}_pow{}.npz')
path2save = join(path_tps, 'rdm_thgh_time/')
tps_save2 = join(path2save, 'TPS_pears_{}_{}_{}_{}_wth_med_split.npz')
###############################################################################
exp = 'Enc' #Ret, Enc
stat = 'pearson'
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs = ['0_theta']
wins = ['early','late']
su_odor_groups = {'CHAF' : {'low':['1','2','4','5'],
                              'high':['3','8','7','9']},
                    'VACJ' : {'low':['11','14','12','10'],
                              'high':['15','17','16','13']},
                    'SEMC' : {'low':['7','10','11','12','13'],
                              'high':['5','8','9']},
                    'PIRJ' : {'low':['1','9','5'],
                              'high':['4','6','7','18']},
                    'LEFC' : {'low':['15','2','1','16'],
                              'high':['14','3','4','17']},
                    'FERJ' : {'low':['7','2','16','17'],
                              'high':['12','1','5','13']},}

#create 2windows in time early and late
for freq in freqs:
    for su in su_odor_groups:
        for cond in su_odor_groups[su]:
            rdm_all_early, rdm_all_late = np.array([]), np.array([])
            early_od, late_od = np.array([]), np.array([])
            for od in su_odor_groups[su][cond]:
                mat = np.load(pow_path.format(su,od,exp[0],freq),allow_pickle=True)
                channels = mat['channel']
                data = mat['xpow'][:,17:47,:]
                nelecs,npts,ntrials = data.shape
                thr = (ntrials/2)
                #check if there is at least 2 trials by odor to create a late array
                if thr < 1:
                    early_od = np.concatenate((early_od,data),axis=-1) if np.size(early_od) else data
                elif thr >= 1 and (thr).is_integer():
                    thr = int(thr)
                    early_od = np.concatenate((early_od,data[:,:,:thr]),axis=-1) if np.size(early_od) else data[:,:,:thr]
                    late_od = np.concatenate((late_od,data[:,:,thr:]),axis=-1) if np.size(late_od) else data[:,:,thr:]
                elif thr >= 1 and not (thr).is_integer():
                    early_od = np.concatenate((early_od,data[:,:,:int(thr-0.5)]),axis=-1) if np.size(early_od) else data[:,:,:int(thr-0.5)]
                    late_od = np.concatenate((late_od,data[:,:,int(thr-0.5):]),axis=-1) if np.size(late_od) else data[:,:,int(thr-0.5):]
            
            print(su,cond,'early', early_od.shape)
            print(su,cond,'late', late_od.shape)
                                                      
            for elec in range(nelecs):
                rdm_early = tpsim_by_cond(early_od[elec,...],stat=stat)
                rdm_late = tpsim_by_cond(late_od[elec,...],stat=stat)
#                 print(rdm_late.shape,rdm_early.shape)
#                 print(rdm_late.shape,rdm_early.shape)
                rdm_all_early = np.concatenate((rdm_all_early,rdm_early),axis=0) if np.size(rdm_all_early) else rdm_early
                rdm_all_late = np.concatenate((rdm_all_late,rdm_late),axis=0) if np.size(rdm_all_late) else rdm_late
            print(su,freq,'R','early rdm',rdm_all_early.shape,'late rdm',rdm_all_late.shape)
            np.savez(tps_save2.format(exp,su,cond,freq[2:],wins[0]),rdm=rdm_all_early,
                    channels=channels)
            np.savez(tps_save2.format(exp,su,cond,freq[2:],wins[1]),rdm=rdm_all_late,
                    channels=channels)
            del rdm_all_early, rdm_all_late
        

In [ ]:
"""
Compute Distance of odors within Poor and Rich conds separately through E and R
Define time windows during E and R --> Dynamics of TPSim
"""
###############################################################################
st = study('Olfacto')
path_tps = join(st.path, 'feature/TPSim_{}_By_Odor_By_Cond/')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor/')
pow_path = join(path_pow, '{}_odor{}_{}_pow{}.npz')
path2save = join(path_tps, 'rdm_thgh_time/')
tps_save2 = join(path2save, 'TPS_spear_{}_{}_{}_{}_2_extr_gob.npz')
###############################################################################
exp = 'Enc' #Ret, Enc
stat = 'pearson'
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs = ['0_theta']
wins = ['early','late']
su_odor_groups = {'CHAF' : {'low':['1','2','4','5'],
                              'high':['3','8','7','9']},
                    'VACJ' : {'low':['11','14','12','10'],
                              'high':['15','17','16','13']},
                    'SEMC' : {'low':['7','10','11','12','13'],
                              'high':['5','8','9']},
                    'PIRJ' : {'low':['1','9','5'],
                              'high':['4','6','7','18']},
                    'LEFC' : {'low':['15','2','1','16'],
                              'high':['14','3','4','17']},
                    'FERJ' : {'low':['7','2','16','17'],
                              'high':['12','1','5','13']},}

#create 2windows in time early and late
for freq in freqs:
    for su in su_odor_groups:
        for cond in su_odor_groups[su]:
            rdm_all_early, rdm_all_late = np.array([]), np.array([])
            early_od, late_od = np.array([]), np.array([])
            for od in su_odor_groups[su][cond]:
                mat = np.load(pow_path.format(su,od,exp[0],freq),allow_pickle=True)
                channels = mat['channel']
                data = mat['xpow'][:,17:47,:]
                nelecs,npts,ntrials = data.shape
                thr = (ntrials/2)
                #check if there is at least 2 trials by odor to compute similarity
                if thr == 1: 
                    early_od = np.concatenate((early_od, data[...,0][...,np.newaxis]),axis=-1) \
                            if np.size(early_od) else data[...,0][...,np.newaxis]
                    late_od = np.concatenate((late_od,data[...,-1][...,np.newaxis]),axis=-1) \
                            if np.size(late_od) else data[...,-1][...,np.newaxis]
                if thr > 1 :
                    early_od = np.concatenate((early_od,data[:,:,:2]),axis=-1) \
                                if np.size(early_od) else data[:,:,:2]
                    late_od = np.concatenate((late_od,data[:,:,-2:]),axis=-1)  \
                                if np.size(late_od) else data[:,:,-2:]
            print(su,cond,'early', early_od.shape)
            print(su,cond,'late', late_od.shape)
                                                      
            for elec in range(nelecs):
                rdm_early = tpsim_by_cond(early_od[elec,...],stat=stat)
                rdm_late = tpsim_by_cond(late_od[elec,...],stat=stat)
#                 print(rdm_late.shape,rdm_early.shape)
#                 print(rdm_late.shape,rdm_early.shape)
                rdm_all_early = np.concatenate((rdm_all_early,rdm_early),axis=0) if np.size(rdm_all_early) else rdm_early
                rdm_all_late = np.concatenate((rdm_all_late,rdm_late),axis=0) if np.size(rdm_all_late) else rdm_late
            print(su,freq,'R','early rdm',rdm_all_early.shape,'late rdm',rdm_all_late.shape)
            np.savez(tps_save2.format(exp,su,cond,freq[2:],wins[0]),rdm=rdm_all_early,
                    channels=channels)
            np.savez(tps_save2.format(exp,su,cond,freq[2:],wins[1]),rdm=rdm_all_late,
                    channels=channels)
            del rdm_all_early, rdm_all_late
        

In [ ]:
from scipy.stats import pearsonr, spearmanr

def tpsim_btw_cond(data0, data1, stat='spearmanr'):
    """compute the Temporal Pattern Similarity (tpsim) for all combinations of
    single-trial power in 2 different conditions
    
    Parameters
    ----------
    data1, data2 : arrays
        Must be of shape (n_pts, n_trials)
    stat : string
        The stat correlation method to use. 'pearson' or 'spearman'
    Returns
    -------
    tpsim : value
        The mean TPSim value 
    """
    corr = pearsonr if stat == 'pearson' else spearmanr
    assert (
        data0.shape[0] == data1.shape[0]
    ), "Error: shape of trial1 and trial2 must have the same npts"
    n_trials0, n_trials1 = data0.shape[1],data1.shape[1]
    
    list_tpsim = np.zeros((n_trials0*n_trials1))
    i = 0
    for trial0, trial1 in product(range(n_trials0),range(n_trials1)):
        list_tpsim[i] += 1-(corr(data0[:,trial0],data1[:,trial1])[0])
        i += 1
    return list_tpsim[np.newaxis]

In [ ]:
"""
Compute Distance btw Poor and Rich through E and R
Define time windows during E and R --> Dynamics of TPSim
"""
###############################################################################
st = study('Olfacto')
path_tps = join(st.path, 'feature/TPSim_{}_By_Odor_By_Cond/')
path_pow = join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_odor/')
pow_path = join(path_pow, '{}_odor{}_{}_pow{}.npz')
path2save = join(path_tps, 'rdm_thgh_time/')
tps_save2 = join(path2save, 'TPS_{}_{}_{}_{}_Low_High_med_split.npz')
###############################################################################
exp = 'Ret' #Ret, Enc
stat = 'pearson'
if not exists(path2save.format(exp)):
    makedirs(path2save.format(exp))
###############################################################################
freqs = ['0_theta']# '1_alpha', '2_beta', '3_gamma']
wins = ['early','late']
su_odor_groups = {'CHAF' : {'low':['1','2','4','5'],
                              'high':['3','8','7','9']},
                    'VACJ' : {'low':['11','14','12','10'],
                              'high':['15','17','16','13']},
                    'SEMC' : {'low':['7','10','11','12','13'],
                              'high':['5','8','9']},
                    'PIRJ' : {'low':['1','9','5'],
                              'high':['4','6','7','18']},
                    'LEFC' : {'low':['15','2','1','16'],
                              'high':['14','3','4','17']},
                    'FERJ' : {'low':['7','2','16','17'],
                              'high':['12','1','5','13']},}

#create 2windows in time early and late
for freq in freqs:
    for su in su_odor_groups:
        rdm_all_early, rdm_all_late = np.array([]), np.array([])
        early_l, early_h, late_l, late_h = np.array([]), np.array([]), np.array([]), np.array([])
        for cond in su_odor_groups[su]:
            early_od, late_od = np.array([]), np.array([])
            for od in su_odor_groups[su][cond]:
                mat = np.load(pow_path.format(su,od,exp[0],freq),allow_pickle=True)
                channels = mat['channel']
                data = mat['xpow'][:,17:47,:]
                print(cond,'data all',data.shape)
                nelecs,npts,ntrials = data.shape
                thr = (ntrials/2)
                #check if there is at least 2 trials by odor to create a late array
                #check if there is an even nb of trials by odor otherwise trials in the middle is included to both arays
                if thr < 1:
                    early_od = np.concatenate((early_od,data),axis=-1) if np.size(early_od) else data
                elif thr >= 1 and (thr).is_integer():
                    thr = int(thr)
                    early_od = np.concatenate((early_od,data[:,:,:thr]),axis=-1) if np.size(early_od) else data[:,:,:thr]
                    print('early',early_od.shape)
                    late_od = np.concatenate((late_od,data[:,:,thr:]),axis=-1) if np.size(late_od) else data[:,:,thr:]
                    print('late',late_od.shape)
                elif thr >= 1 and not (thr).is_integer():
                    early_od = np.concatenate((early_od,data[:,:,:int(thr-0.5)]),axis=-1) if np.size(early_od) else data[:,:,:int(thr-0.5)]
                    late_od = np.concatenate((late_od,data[:,:,int(thr-0.5):]),axis=-1) if np.size(late_od) else data[:,:,int(thr-0.5):]
            if cond == 'low':
                early_l, late_l = early_od, late_od
            elif cond == 'high':
                early_h, late_h = early_od, late_od
        #print(early_l.shape,early_h.shape)
        #print(late_l.shape,late_h.shape)
        for elec in range(nelecs):
            rdm_early = tpsim_btw_cond(early_l[elec,...],early_h[elec,...],stat=stat)
            rdm_late = tpsim_btw_cond(late_l[elec,...],late_h[elec,...],stat=stat)
            rdm_all_early = np.concatenate((rdm_all_early,rdm_early),axis=0) if np.size(rdm_all_early) else rdm_early
            rdm_all_late = np.concatenate((rdm_all_late,rdm_late),axis=0) if np.size(rdm_all_late) else rdm_late
        print(su,freq,'TPSim','early rdm',rdm_all_early.shape,'late rdm',rdm_all_late.shape)
        np.savez(tps_save2.format(exp,stat,su,freq[2:],wins[0]),rdm=rdm_all_early,
                channels=channels)
        np.savez(tps_save2.format(exp,stat,su,freq[2:],wins[1]),rdm=rdm_all_late,
                channels=channels)
        del rdm_all_early, rdm_all_late
        